Yuanhao Zuo
933643293

# CSX46 Homework Assignment 2: Network Motif Analysis

Load in the *E. coli* operon-operon network (gene regulatory network) data in edge-list format; display the first six lines of data as a sanity check. Use `pandas.read_csv` and `pandas.DataFrame.head`.

In [78]:
import pandas
import igraph
import numpy as np
import matplotlib.pyplot as plt

bacterial_gene_data  = pandas.read_csv("shared/ecolitfnet.txt", sep="\t", names=["source","target"]).drop_duplicates()
pandas.DataFrame.head(bacterial_gene_data)


,source,target
0,ADA,AIDB
1,ADIY,GADX
2,ALLR,ALLS
3,ARCA,BETI
4,ARCA,DPIA


Make an igraph directed graph from the network; print a graph summary as a sanity check.  You will need to use `igraph.Graph.TupleList` and `igraph.Graph.summary`.

In [79]:
from igraph import Graph

bacterial_gene_graph = Graph.TupleList(bacterial_gene_data[["source","target"]].values.tolist(),
                                 directed=True)
bacterial_gene_graph.summary()

'IGRAPH DN-- 133 261 -- \n+ attr: name (v)'

## Part I: Three-vertex motif analysis

Obtain the counts of all three-vertex motifs in the graph. You will need to use the `motifs_randesu` method in `igraph.Graph`.

*Question 1: Which one of these motifs is most frequent in the E. coli regulatory network?*

In [54]:
a = bacterial_gene_graph.motifs_randesu(size = 3, cut_prob = None, callback = None)
b = numpy.nanmax(a)
bacterial_gene_graph.motifs_randesu(size = 3, cut_prob = None, callback = None).index(b)

6

HINT: The most frequent motif has 1,676 occurrences in the network.

*Question 2: Which one of these motifs has a count of 47 in the regulatory network? I’ll refer to this specific motif as the “mystery motif”, or MM.*

In [37]:
bacterial_gene_graph.motifs_randesu(size = 3, cut_prob = None, callback = None).index(47)


7

Get motif counts for 10,000 different randomizations of the network.  You will need to use a `for` loop, `igraph.Graph.copy`, `igraph.Graph.rewire`, and `append`.  You can specify n=1000 for the rewiring.

In [46]:
bacterial_gene_graph_copy = bacterial_gene_graph.copy()
MM_vector = []
for i in range(10000):
    bacterial_gene_graph_copy.rewire(1000)
    MM_vector.append(bacterial_gene_graph_copy.motifs_randesu(size = 3)[7])
print(MM_vector)

[59, 60, 60, 59, 53, 49, 59, 65, 55, 64, 80, 57, 57, 49, 58, 59, 50, 61, 47, 75, 46, 55, 49, 62, 58, 57, 52, 72, 53, 61, 36, 44, 62, 47, 46, 65, 52, 61, 66, 70, 73, 61, 64, 73, 62, 76, 70, 68, 54, 54, 59, 57, 65, 68, 52, 67, 79, 56, 80, 62, 67, 45, 58, 53, 72, 49, 74, 62, 62, 60, 57, 53, 49, 72, 66, 50, 62, 59, 49, 48, 77, 48, 54, 51, 62, 54, 73, 59, 53, 47, 57, 47, 50, 65, 68, 59, 50, 56, 47, 66, 47, 49, 60, 74, 58, 60, 53, 61, 80, 58, 41, 48, 62, 51, 50, 42, 70, 46, 55, 62, 62, 47, 53, 43, 61, 53, 74, 58, 58, 57, 57, 57, 74, 54, 65, 54, 47, 60, 49, 58, 52, 53, 58, 68, 60, 61, 57, 58, 66, 63, 58, 62, 57, 47, 62, 67, 48, 66, 56, 61, 72, 61, 71, 55, 63, 62, 70, 44, 54, 68, 54, 53, 65, 73, 65, 55, 49, 66, 60, 54, 59, 50, 43, 59, 53, 66, 52, 60, 65, 68, 65, 61, 51, 68, 65, 61, 51, 46, 66, 61, 71, 63, 61, 47, 61, 57, 65, 70, 52, 43, 60, 69, 70, 66, 50, 63, 52, 82, 71, 54, 75, 42, 58, 81, 71, 50, 61, 51, 52, 49, 58, 62, 59, 76, 55, 59, 60, 61, 65, 60, 59, 65, 51, 45, 52, 68, 67, 62, 61, 65,

For each motif type, compute the mean and standard deviation of the counts of appearances of the motif in each of the 10,000 randomly rewired graphs. You will need to use the `numpy.mean` and `numpy.std` functions.

In [66]:
mean_MM_vector = numpy.mean(MM_vector)
sd_MM_vector = numpy.std(MM_vector)

For each motif type, obtain the *Z* score for the number of appearances of the motif in the real network, vs. in the randomly rewired networks; store this in object `motif_counts_zscores`.

In [67]:
motif_counts_zscores = (47 - numpy.mean(MM_vector))/ numpy.std(MM_vector)

*Question:  What are the mean and standard deviation of the MM counts for the 10,000 random networks?*

In [68]:
print("The mean of the counts is :", mean_MM_vector)
print("The standard deviation of the counts is :", " %.4f " % sd_MM_vector)

the mean of the counts is : 58.0053
the standard deviation of the counts is :  9.3802 


*Question 3: What is the Z-score for enrichment of the MM count for the real network vs. the ensemble of random networks? Does this z-score correspond to a statistically significant positive enrichment?*

In [72]:
print("The Z-score for enrichment of the MM count for the real network is:", " %.4f " % motif_counts_zscores)

The Z-score for enrichment of the MM count for the real network is:  -1.1732 


No, it is a negative value.

*Question 4: What is the ratio of the MM count for the real network to the average MM count for the random networks?*

In [74]:
ratio = 47/ mean_MM_vector
print("The ratio of the MM is:",ratio)

The ratio of the MM is: 0.8102707856006262


*Question 5: How does ratio compare to the same ratio for the data in Table 1 in Shen-Orr et al., Nature Genetics, 2002?*

The ratio is smaller than the same ratio in the article.

*Question 6: Given the modest ratio of the MM frequency in the real network vs. randomly shuffled network, should we entertain the possibility that the high frequency of MMs in the real network could be a consequence of the degree distribution rather than evolution specifically “favoring” FFLs as a building block for gene regulatory networks?*

## Part II:  Four-vertex motif analysis

*Question 1: What is the count of this four-vertex motif in the E. coli gene regulatory network?*

Figure out the isomorphism class of the network shown in the assignment PDF.  You'll want to build an edge-list tuple-list and then an igraph `Graph` object using `Graph.TupleList` (make sure to specify a directed graph!), and then use the `isoclass` method.

In [80]:
four_vertex_data = {"source":["H", "H", "GADW", "GADW"],
        "target":["GADE","GADX","GADE","GADX"]}
iso_df = pandas.DataFrame(four_vertex_data)
iso_graph = Graph.TupleList(iso_df[["source","target"]].values.tolist(),
                                 directed=True)
iso_graph.isoclass()

19

Compute the number of appearances of this four-vertex motif in the real network, using `motifs` with `size=4`.

In [85]:
gene_motifs_4_amount = bacterial_gene_graph.motifs_randesu(size = 4, cut_prob = None, callback = None)
print(gene_motifs_4_amount[19])

297


*Question 2: What are the average (and standard deviation) of the counts of this motif in 10,000 randomly shuffled networks?*

Get the mean and standard deviation for the count of four-vertex motifs in 10,000 randomly rewired graphs.  You will need to use a `for` loop, `igraph.Graph.copy`, `igraph.graph.rewire`, and `append`.  You can specify n=1000 for the rewiring.

In [86]:
bacterial_gene_graph_copy = bacterial_gene_graph.copy()
gene_vector_4 = []
for i in range(10000):
    bacterial_gene_graph_copy.rewire(1000)
    gene_vector_4.append(bacterial_gene_graph_copy.motifs_randesu(size = 4)[19])
print(gene_vector_4)

[152, 133, 183, 304, 152, 99, 154, 186, 154, 168, 176, 191, 200, 166, 167, 171, 219, 296, 214, 87, 171, 159, 189, 219, 164, 214, 130, 137, 159, 206, 194, 216, 134, 130, 111, 177, 192, 212, 162, 197, 185, 172, 185, 235, 265, 139, 162, 153, 184, 190, 142, 261, 155, 220, 191, 190, 211, 144, 98, 205, 128, 118, 169, 135, 185, 182, 166, 136, 171, 218, 176, 193, 155, 190, 166, 198, 152, 204, 206, 217, 155, 148, 165, 202, 130, 133, 179, 188, 204, 234, 166, 198, 188, 118, 144, 246, 234, 252, 244, 193, 210, 193, 174, 237, 205, 187, 185, 146, 134, 90, 170, 108, 202, 175, 268, 177, 217, 166, 186, 221, 205, 157, 178, 164, 206, 180, 125, 134, 204, 200, 108, 198, 198, 212, 213, 181, 177, 140, 226, 206, 219, 181, 120, 140, 145, 107, 239, 153, 138, 111, 176, 139, 170, 189, 202, 125, 173, 181, 130, 189, 115, 132, 131, 251, 176, 172, 118, 193, 151, 142, 152, 187, 141, 212, 139, 206, 115, 219, 135, 164, 112, 212, 123, 177, 213, 180, 212, 304, 126, 183, 212, 198, 154, 164, 196, 181, 169, 143, 192, 154, 136

Print out the mean number of 4-vertex DOR motifs for the random networks, the standard deviation, and the Z score for the DOR motif count in the real graph vs. the random graphs. You'll need to use `numpy.array`, `numpy.mean`, and `numpy.std`.

In [87]:
mean_vector_4 = numpy.mean(gene_vector_4)
sd_vector_4 = numpy.std(gene_vector_4)
print("The mean of the counts is :", mean_vector_4)
print("The standard deviation of the counts is :", " %.4f " % sd_vector_4)

The mean of the counts is : 177.2472
The standard deviation of the counts is :  38.0521 


*Question 3: What is the Z-score for enrichment of this four-vertex motif?  Is it significant?*

In [92]:
motif_4_counts_zscores = (gene_motifs_4_amount[19]- mean_vector_4) / sd_vector_4
print("The Z-score for enrichment of the four-vertex count for the real network is:", " %.4f " % motif_4_counts_zscores)

The Z-score for enrichment of the four-vertex count for the real network is:  3.1471 


You may find `scipy.stats.norm.cdf` useful

In [94]:
from scipy.stats import norm
print(norm.cdf(gene_motifs_4_amount[19],mean_vector_4, sd_vector_4))

0.9991754327933282


*Question 4: What is the ratio of the count in the real network to the average count for the random networks?  How does this ratio compare to the ratio that you obtain from Table 1 from Shen-Orr et al.? Are they consistent? Does this suggest that Shen-Orr’s actual network randomization procedure is possibly not consistent with their description in the Methods section of their paper, i.e., that it may have had some kind of error?*

In [88]:
ratio_4 = gene_motifs_4_amount[19] / mean_vector_4
print(ratio_4)

1.6756259055150096


The ratio is smaller than the one in article. No,that does not mean Shen-Orr’s procedure is not consistent, I think it is because the multiple input and output in DOR, and the overlapping situation may affect that.